In [1]:
import pandas as pd
import numpy as np
import sys

sys.path.append("/Users/sva21/Documents/mangrove/code/backtester-RL/")

## Orders


In [2]:
import src.order

In [3]:
order_type = "bid"

qty_b1 = 100
price_b1 = 99
bid1 = src.order.Order(order_type, qty_b1, price_b1)

qty_b2 = 10
price_b2 = 98
bid2 = src.order.Order(order_type, qty_b2, price_b2)

order_type = "ask"

qty_a1 = 100
price_a1 = 101
ask1 = src.order.Order(order_type, qty_a1, price_a1)

qty_a2 = 10
price_a2 = 102
ask2 = src.order.Order(order_type, qty_a2, price_a2)

## Order Book


In [4]:
from src.order_book import OrderBook

orderbook = OrderBook([bid1, bid2], [ask1, ask2])
print(orderbook)

┌─────────┬────────┬─────────┬─────────┐
│   Index │   Size │   Price │   Total │
├─────────┼────────┼─────────┼─────────┤
│       0 │     10 │     102 │    1020 │
│       1 │    100 │     101 │   10100 │
│       2 │    100 │      99 │    9900 │
│       3 │     10 │      98 │     980 │
└─────────┴────────┴─────────┴─────────┘


## Main Script


In [5]:
from src.kandel import kandel_simulator
from src.time_series import load_csv

In [6]:
ts = load_csv("data/ETHUSDC-1s-2024-08-clean.csv")

In [39]:
window = 3600 * 12
quote = 37500 # USDC
base = 37500 # also in USDC, will be swapped to base at day 0
vol_mult = 1.645
n_points = 10
step_size = 1
days = 10

transactions, res, order_book_history = kandel_simulator(
    ts=ts[-(days + 1) * 24 * 3600 :],
    quote=quote,
    base=base,
    vol_mult=vol_mult,
    n_points=n_points,
    step_size=step_size,
    window=window,
)
res.to_pandas().to_csv("results/simul_results.csv")

100%|██████████| 864000/864000 [02:48<00:00, 5114.28it/s]


- Benchmark sur position value ETH ✔
- Shift la window en dehors des jours de backtests ✔
- Amélirorer le graph de PNL ✔
- Permettre d'avoir une strat d'exit/reentry
- Plot l'évolution de la position dans le temps

- tester si on se base l'IV au lieu de la volat daily et comparer
- impact du multiplieur
- impact du nombre de bid ask set up au ddébut

- comment reposition les bids / ask après une exécution de ask / bid


In [40]:
for i,transac  in enumerate(transactions):
    if transac:
        print(transac, i)

[bid 1.43136@2619.88537] 8206
[bid 1.437815@2608.123781] 10131
[bid 1.444299@2596.414993] 10300
[ask 1.444299@2608.123781] 11412
[ask 1.437815@2619.88537] 15996
[ask 1.43136@2631.7] 25403
[ask 1.424934@2643.567909] 39512
[bid 1.43136@2631.7] 45178
[bid 1.437815@2619.88537] 47284
[bid 1.444299@2608.123781] 50912
[bid 1.450812@2596.414993] 55393
[ask 1.450812@2608.123781] 58852
[ask 1.444299@2619.88537] 75408
[ask 1.431278@2632.60105] 87820
[ask 1.431278@2641.451757] 88649
[ask 1.431278@2650.332219] 88704
[bid 1.43609@2641.451757] 89520
[ask 1.43609@2650.332219] 105072
[ask 1.431278@2659.242537] 105343
[ask 1.431278@2668.182811] 105493
[ask 1.431278@2677.153142] 105974
[bid 1.43609@2668.182811] 106561
[ask 1.43609@2677.153142] 108011
[ask 1.431278@2686.153631] 110997
[bid 1.43609@2677.153142] 112761
[bid 1.440918@2668.182811] 114605
[ask 1.440918@2677.153142] 118986
[bid 1.445762@2668.182811] 121531
[bid 1.43609@2659.242537] 126528
[ask 1.43609@2668.182811] 132574
[bid 1.440918@2659.2425

# Results


In [41]:
import pandas as pd  # type: ignore

pd.options.plotting.backend = "plotly"

from plotly.subplots import make_subplots
import plotly.graph_objects as go
from tqdm import tqdm

In [42]:
res = pd.read_csv("results/simul_results.csv", index_col=0, parse_dates=True)
prices = res[["price"]].copy()

In [43]:
res[window-1:]

,price,quote,base,mtm,volume,uptime
time,,,,,,
2024-08-21 23:59:59,2631.70,37500.000000,14.249345,75000.000000,0.0,0.0
2024-08-22 00:00:00,2631.70,75000.000000,0.000000,75000.000000,0.0,1.0
2024-08-22 00:00:01,2631.70,75000.000000,0.000000,75000.000000,0.0,1.0
2024-08-22 00:00:02,2631.70,75000.000000,0.000000,75000.000000,0.0,1.0
2024-08-22 00:00:03,2631.70,75000.000000,0.000000,75000.000000,0.0,1.0
...,...,...,...,...,...,...
2024-08-31 23:59:55,2513.13,69285.669862,1.466288,72970.642223,0.0,1.0
2024-08-31 23:59:56,2513.13,69285.669862,1.466288,72970.642223,0.0,1.0
2024-08-31 23:59:57,2513.13,69285.669862,1.466288,72970.642223,0.0,1.0


In [44]:
res["mtm_eth"] = res["mtm"] / res["price"]
res["base_returns"] = res["mtm_eth"] / ((quote + base) / res["price"][window]) -1
res["quote_returns"] = res["mtm"] / (quote + base)-1

/var/folders/kn/wgl1sqns1bxgynxc599_5y9c0000gp/T/ipykernel_64339/2563357684.py:2: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [45]:
window_hours = window // 3600 - 1
res_1h = res.resample("1h").last()

# PLOT
prices_res = prices.resample("1h").last()

fig = make_subplots(
    rows=3,
    cols=1,
    shared_xaxes=True,
    specs=[[{"secondary_y": True}], [{}], [{}]],
    vertical_spacing=0.05,
    subplot_titles=("MTM", "Returns", "Price"),
)
fig.add_trace(
    go.Scatter(
        x=res_1h.index[window_hours:],
        y=res_1h.mtm[window_hours:],
        mode="lines",
        name="MTM in USDC",
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=res_1h.index[window_hours:],
        y=res_1h.mtm_eth[window_hours:],
        mode="lines",
        name="MTM in ETH",
    ),
    row=1,
    col=1,
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(
        x=res_1h.index[window_hours:],
        y=res_1h.quote_returns[window_hours:],
        mode="lines",
        name="Over holding USDC",
    ),
    row=2,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=res_1h.index[window_hours:],
        y=res_1h.base_returns[window_hours:],
        mode="lines",
        name="Over holding ETH",
    ),
    row=2,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=res_1h.index[window_hours:],
        y=res_1h.price[window_hours:],
        mode="lines",
        name="Price",
    ),
    row=3,
    col=1,
)

fig.update_layout(
    height=1000,
    width=1400,
    title_text=f"MTM, Returns and Price.<br><sup>window = {window} | quote = {quote} | base = {base} | vol_mult = {vol_mult} | n_points = {n_points} | step_size = {step_size} | days = {days}</sup>",
)

fig.update_yaxes(row=2, col=1, tickformat=".2%")

fig.show()

fig.write_image(
    f"results/saves/{window}_{quote}_{base}_{vol_mult}_{n_points}_{step_size}_{days}.png",
    width=1400,
    height=1000,
)

In [46]:
order_book_history_df = pd.DataFrame.from_records([ob.to_dict()  for ob in order_book_history])
order_book_history_df.index = res[window-1:].index
order_book_history_1h = order_book_history_df.resample("1h").last()

In [47]:
order_book_history_1h.head()

,bids,asks
time,,
2024-08-21 23:00:00,"[2619.88537, 2608.123781, 2596.414993, 2584.75...","[2643.567909, 2655.489337, 2667.464526, 2679.4..."
2024-08-22 00:00:00,"[2619.88537, 2608.123781, 2596.414993, 2584.75...","[2643.567909, 2655.489337, 2667.464526, 2679.4..."
2024-08-22 01:00:00,"[2619.88537, 2608.123781, 2596.414993, 2584.75...","[2643.567909, 2655.489337, 2667.464526, 2679.4..."
2024-08-22 02:00:00,"[2584.758771, 2573.154877, 2561.603077, 2550.1...","[2608.123781, 2619.88537, 2631.7, 2643.567909,..."
2024-08-22 03:00:00,"[2596.414993, 2584.758771, 2573.154877, 2561.6...","[2619.88537, 2631.7, 2643.567909, 2655.489337,..."


In [48]:
fig = make_subplots(
    rows=1,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=(""),
)

for i, ob in enumerate(order_book_history_1h.iterrows()):
    bids = ob[1].bids
    asks = ob[1].asks

    fig.add_trace(
        go.Scatter(
            x=[i for _ in range(len(bids))],
            y=tuple(bids),
            mode="markers",
            name="Bids",
            showlegend=False,
            marker=dict(color="green", symbol="line-ew-open", size=8, line=dict(width=5, color="black")),
        ),
    )

    fig.add_trace(
        go.Scatter(
            x=[i for _ in range(len(asks))],
            y=tuple(asks),
            mode="markers",
            name="Asks",
            showlegend=False,
            marker=dict(color="red", symbol="line-ew-open", size=8, line=dict(width=5, color="black")),
        ),
    )

fig.add_trace(
    go.Scatter(
        x=[i for i in range(len(order_book_history_1h))],
        y=res_1h.price[window_hours:],
        mode="lines",
        name="Price",
        marker=dict(color="blue"),
    )
)

fig.update_layout(
    height=1000,
    width=1600,
    title_text=f"Order book<br><sup>window = {window} | quote = {quote} | base = {base} | vol_mult = {vol_mult} | n_points = {n_points} | step_size = {step_size} | days = {days}</sup>",
)

fig.show()